In [1]:
import numpy as np

def probality_to_Rx(probability: float):
    return 2 * np.arcsin(np.sqrt(probability))

In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from numpy import pi

# Use Aer's AerSimulator
simulator = AerSimulator()

# Create a Quantum Circuit acting on the q register
circuit = QuantumCircuit(10, 10)

# P(flooding) = max(exp(4*hsea – 2))
h_sea = 0.1
P_flooding = min(1, np.e ** (4 * h_sea - 2))

# P(flooding area|flooding) = max(1, exp( 2*[hsea–harea] ) )
f = lambda h_area : min(1, np.e ** (2 * (h_sea - h_area)))
h_area_river_shores = 0.5
h_area_waterfront = 1
h_area_shopping = 2
h_area_tourist = 2
h_area_recreational = 1.5
P_flooding_river_shores = f(h_area_river_shores)
P_flooding_waterfront = f(h_area_waterfront)
P_flooding_shopping = f(h_area_shopping)
P_flooding_tourist = f(h_area_tourist)
P_flooding_recreational = f(h_area_tourist)

circuit.rx(probality_to_Rx(P_flooding), 0)                   # 0 Flooding
circuit.crx(probality_to_Rx(P_flooding_river_shores), 0, 1)  # 1 * River shores flooding
circuit.crx(probality_to_Rx(P_flooding_waterfront), 0, 2)    # 2 * Waterfront flooding
circuit.crx(probality_to_Rx(0.75), 2, 3)                     # 3 * * 75% Flooding of the promenade
circuit.crx(probality_to_Rx(0.50), 2, 4)                     # 4 * * 50% Flooding the yard
circuit.crx(probality_to_Rx(P_flooding_shopping), 0, 5)      # 5 * Flooding of shopping area
circuit.crx(probality_to_Rx(0.80), 5, 6)                     # 6 * * 80% Buildings inundation
circuit.crx(probality_to_Rx(P_flooding_tourist), 0, 7)       # 7 * Flooding of tourist area
circuit.crx(probality_to_Rx(0.60), 7, 8)                     # 8 * * 60% Destruction of hotels
circuit.crx(probality_to_Rx(P_flooding_recreational), 0, 9)  # 9 * Flooding of recreational areas

# Add a CX (CNOT) gate on control qubit 0 and target qubit 1

# Map the quantum measurement to the classical bits
circuit.barrier()
circuit.measure(list(range(10)), list(range(10)))

# Compile the circuit for the support instruction set (basis_gates)
# and topology (coupling_map) of the backend
compiled_circuit = transpile(circuit, simulator)

# Execute the circuit on the aer simulator
shots = 1000
job = simulator.run(compiled_circuit, shots=shots)

# Grab results from the job
result = job.result()

# Returns counts
counts = result.get_counts(compiled_circuit)

# Draw the circuit
# circuit.draw("mpl")

In [3]:
def counts_to_probabilities(counts):
    d = dict()
    registers = len(list(counts.keys())[0])
    for i in range(registers):
        d[i] = 0
    for k, v in counts.items():
        for i, xd in enumerate(k):
            if k[i] == '1':
                d[registers - i - 1] += v
    return d


props = counts_to_probabilities(counts)
for i in props:
    props[i] /= shots / 100

props

{0: 20.1,
 1: 8.5,
 2: 3.6,
 3: 2.8,
 4: 1.5,
 5: 0.4,
 6: 0.4,
 7: 0.8,
 8: 0.4,
 9: 0.5}

In [4]:
names = [
    'Flooding',
    'River shores flooding',
    'Waterfront flooding',
    'Flooding of the promenade',
    'Flooding the yard',
    'Flooding of shopping area',
    'Buildings inundation',
    'Flooding of tourist area',
    'Destruction of hotels',
    'Flooding of recreational areas'
]

props2 = dict()
for k, v in props.items():
    props2[names[k]] = v / 100

props2

{'Flooding': 0.201,
 'River shores flooding': 0.085,
 'Waterfront flooding': 0.036000000000000004,
 'Flooding of the promenade': 0.027999999999999997,
 'Flooding the yard': 0.015,
 'Flooding of shopping area': 0.004,
 'Buildings inundation': 0.004,
 'Flooding of tourist area': 0.008,
 'Destruction of hotels': 0.004,
 'Flooding of recreational areas': 0.005}

In [5]:
costs_per_area = {
    'Flooding': 200,
    'River shores flooding': 70,
    'Waterfront flooding': 150,
    'Flooding of the promenade': 240,
    'Flooding the yard': 950,
    'Flooding of shopping area': 170,
    'Buildings inundation': 800,
    'Flooding of tourist area': 200,
    'Destruction of hotels': 1_200,
    'Flooding of recreational areas': 500
}

expected_costs = 0
for k, v in props2.items():
    expected_costs += costs_per_area[k] * v

print(f'Cost: {expected_costs} mln zł!!!!')

Cost: 85.30000000000001 mln zł!!!!
